# New Measure - Aspirin EC 75mg tablets per 1000 patients

## Rationale

Patients sometimes get prescribed aspirin with a "gastro-resistant" coating (also called enteric coating or EC) in order to protect from the gastro-intestinal side-effects associated with aspirin, by stopping the tablet dissolving in the stomach, and instead dissolving in the higher pH duodenum.

However, according to UKMI, there is only weak evidence to suggest reduced blood loss or decreased GI ulceration with EC aspirin, and that it is insufficient to support a clinical benefit over standard release (plain) aspirin.  In addition, there is speculation that enteric coating reduces the anti-platelet effect of aspirin, and thus its efficacy in the prevention of cardiovascular events. The available evidence for this is conflicting. There is a theory of pseudoresistance reflecting delayed and reduced drug absorption with EC aspirin over normal plain aspirin.

Therefore, EC aspirin should not be routinely used in preference to uncoated aspirin products.




## Proposed Measure

**Name:** Aspirin enteric-coated 75mg tablets prescribing per 1000 patients

**Description:** Prescribing of aspirin enteric-coated 75mg tablet per 1000 registered patients 

**Why It Matters**: 

Patients sometimes get prescribed aspirin with a "gastro-resistant" coating (also called enteric coating or EC) in order to protect from the gastro-intestinal side-effects associated with aspirin, by stopping the tablet dissolving in the stomach, and instead dissolving in the higher pH duodenum.

However, according to UKMI, there is only weak evidence to suggest reduced blood loss or decreased GI ulceration with EC aspirin, and that it is insufficient to support a clinical benefit over standard release (plain) aspirin.  In addition, there is speculation that enteric coating reduces the anti-platelet effect of aspirin, and thus its efficacy in the prevention of cardiovascular events. The available evidence for this is conflicting. There is a theory of pseudoresistance reflecting delayed and reduced drug absorption with EC aspirin over normal plain aspirin.

Therefore, EC aspirin should not be routinely used in preference to uncoated aspirin products.


### Numerators and Denominators

Numerator: `bnf_code LIKE '0209000A0%AK' --Aspirin Tab_E/C 75mg`

Denominator: patient list size



## SQL and results

In [1]:
#import libraries required for analysis
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ebmdatalab import bq
from ebmdatalab import charts
from ebmdatalab import maps

In [2]:
#calculate the number of items per 1000 patients for aspirin EC 75mg tabs (and brands)
sql = """
SELECT rx.month, rx.pct, sum(rx.items) as items, total_list_size, 
1000*sum(ieee_divide(rx.items,total_list_size)) as measure --calculates number of items and items per 1000 patients
FROM
hscic.normalised_prescribing_standard as rx
INNER JOIN (
SELECT
  prac.ccg_id as ccg,
  month,
  SUM(total_list_size) AS total_list_size
FROM
  hscic.practice_statistics AS prac_stats
INNER JOIN
  hscic.practices AS prac
ON
prac.code=prac_stats.practice
INNER JOIN
  hscic.ccgs as ccgs
ON
prac.ccg_id = ccgs.code
WHERE
ccgs.org_type = 'CCG'
GROUP BY
  month,
  prac.ccg_id) AS stats --this creates the list size for a CCG, and ensures that only CCGs are included in the data
ON
  rx.month=stats.month
  AND rx.pct=stats.ccg
WHERE 
bnf_code LIKE '0209000A0%AK' --Aspirin Tab_E/C 75mg (brand and generic)
GROUP BY rx.month, rx.pct, stats.total_list_size
ORDER BY month
"""
asp_ec__df = bq.cached_read(sql, csv_path='asp_ec_df.csv', use_cache=True)
asp_ec_df['month'] = asp_ec_df['month'].astype('datetime64[ns]')


NameError: name 'asp_ec_df' is not defined

## Changes in prescribing levels

In [ ]:
#plot data on graph
asp_ec_df.groupby(["month"])['items'].sum().plot(kind='line', title="Total items for aspirin EC 75mg tablets in England")

This shows that, although reducing, there has been significant prescribing of aspirin EC 75mg tabs in the last five years.

## Modelling for measure

In [ ]:
#create sample deciles
charts.deciles_chart(
        asp_ec_df,
        period_column='month',
        column='measure',
        title="Aspirin EC 75mg tablet items per 1000 patients",
        show_outer_percentiles=True)

#add in example CCG (Devon - 15N)
df_subject = asp_ec_df.loc[asp_ec_df['pct'] == '15N']
plt.plot(df_subject['month'], df_subject['measure'], 'r--')
plt.show()

There is significant use in some CCGs, along with significant variation.

In [ ]:
#create choropeth map of aspirin 75mg EC per 1000 patients
plt.figure(figsize=(12, 7))
latest_per_1000_df = asp_ec_df.loc[asp_ec_df['month'] == '2019-10-01']
plt = maps.ccg_map(latest_per_1000_df, title="Aspirin EC 75mg tablet items per 1000 patients\n(October 2019)", column='measure', cartogram=False, separate_london=True)
plt.show()

## Proposed JSON

```{
  "name": "Aspirin 75mg EC tablets",
  "title": [
    "Aspirin 75mg EC tablets prescribing per 1000 patients"
  ],
  "description": [
    "Prescribing of aspirin 75mg EC tablets per 1000 registered patients"
  ],
  "why_it_matters": [
    "<p>Patients sometimes get prescribed aspirin with a 'gastro-resistant' coating (also called enteric coating or EC) in ",
    "order to protect from the gastro-intestinal side-effects associated with aspirin, by stopping the tablet dissolving ",
    "in the stomach, and instead dissolving in the higher pH duodenum.</p>",
    "<p>However, <a href='https://www.sps.nhs.uk/articles/is-there-evidence-to-support-the-use-of-enteric-coated-ec-aspirin-to-reduce-gastrointestinal-side-effects-in-patients-with-cardiovascular-disease/'>  ",
    "according to UKMI</a>, there is only weak evidence to suggest reduced blood loss or decreased GI ulceration ",
    "with EC aspirin, and that it is insufficient to support a clinical benefit over standard release (plain) aspirin. ",
    "In addition, there is speculation that enteric coating reduces the anti-platelet effect of aspirin, and thus its ",
    "efficacy in the prevention of cardiovascular events. The available evidence for this is conflicting. There is a ",
    "theory of pseudoresistance reflecting delayed and reduced drug absorption with EC aspirin over normal plain aspirin.</p>",
    "<p>Therefore, EC aspirin should not be routinely used in preference to uncoated aspirin products.</p>"
  ],
  "numerator_short": "Vitamin B complex items",
  "denominator_short": "1000 patients",
  "url": null,
  "is_percentage": false,
  "is_cost_based": false,
  "low_is_good": true,
  "tags": [
    "core",
    "efficacy",
    "nice"
  ],
  "numerator_type": "bnf_items",
  "numerator_where": [
    "bnf_code LIKE '0209000A0%AK' --Aspirin Tab_E/C 75mg (brand and generic)"
  ],
  "denominator_type": "list_size",
  "date_reviewed": [
    ""
  ],
  "next_review": [
    ""
  ],
  "authored_by": [
    "richard.croker@phc.ox.ac.uk"
  ],
  "checked_by": [
    ""
  ],
  "measure_complexity": [
    "low"
  ],
  "measure_notebook_url": [
    ""
  ]
}```